In [7]:
# ==============================================================================
# NOTEBOOK DE TESTES METAMÓRFICOS - NÍVEL 1: ANÁLISE DE SENSIBILIDADE
# ==============================================================================
# Objetivo: Avaliar a estabilidade e a sensibilidade do sistema a perturbações,
# ausências e valores extremos nos dados de entrada (caixa-preta estrita).
# ------------------------------------------------------------------------------

# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from pathlib import Path

# ======================================================================
# 📂 OUTPUT DIRECTORY (FIGURES & REPORTS)
# ======================================================================
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ==============================================================================
# 📁 ETAPA 1: SETUP DO AMBIENTE
# ==============================================================================

ethanol_path = Path(
    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/trackLog-2023-Feb-13_06-38-49_seg.csv"
)
gasoline_path = Path(
    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/trackLog-2022-Dec-01_06-43-57_qui.csv"
)

df_ethanol = pd.read_csv(ethanol_path)
df_gasoline = pd.read_csv(gasoline_path)

print("Arquivos de dados carregados com sucesso.")

# ==============================================================================
# 🔧 PREPARAÇÃO DOS DADOS
# ==============================================================================

def preparar_dados(df):
    df = df.copy()
    df.columns = df.columns.str.strip()

    col_map = {
        "Speed (OBD)(km/h)": "Speed(OBD)(km/h)",
        "Latitude ": "Latitude",
        "Longitude ": "Longitude"
    }
    df.rename(columns={k: v for k, v in col_map.items() if k in df.columns}, inplace=True)

    if "Speed(OBD)(km/h)" in df.columns:
        speed_ms = df["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df["Acceleration"] = np.diff(speed_ms, prepend=speed_ms[0])

    return df

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# ==============================================================================
# 🤖 CARREGAMENTO DOS MODELOS
# ==============================================================================
# Os modelos foram previamente treinados e validados fora deste notebook.
# Neste experimento, eles são utilizados exclusivamente para avaliação metamórfica.

modelos = {
    "ethanol": {
        "afr": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_ethanol_afr.pkl")
        ),
        "maf": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_ethanol_maf.pkl")
        ),
    },
    "gasoline": {
        "afr": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_gasoline_afr.pkl")
        ),
        "maf": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_gasoline_maf.pkl")
        ),
    }
}

print("Modelos carregados com sucesso.")

# ==============================================================================
# 🧮 FUNÇÕES AUXILIARES
# ==============================================================================

def compute_co2(afr, maf, fuel):
    afr = np.asarray(afr)
    maf = np.asarray(maf)
    carbon_fraction = {"ethanol": 0.5217, "gasoline": 0.8571}
    fuel_rate = np.divide(maf, afr, out=np.zeros_like(maf), where=afr != 0)
    return fuel_rate * carbon_fraction[fuel] * (44.0 / 12.0)

def encontrar_coluna(df, nomes):
    for n in nomes:
        if n in df.columns:
            return df[n]
    return None

def pearson_safe(x, y):
    x, y = np.asarray(x), np.asarray(y)
    mask = ~np.isnan(x) & ~np.isnan(y)
    if mask.sum() < 2:
        return np.nan
    return pearsonr(x[mask], y[mask])[0]

# ==============================================================================
# 🔬 TESTE METAMÓRFICO NUMÉRICO – NÍVEL 1
# ==============================================================================

def executar_teste_plotly(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_ct):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for f in features:
        if f not in df_original.columns:
            df_original[f] = 0
        if f not in df_modificado.columns:
            df_modificado[f] = 0

    # ------------------------------------------------------------------
    # Ground Truth
    # ------------------------------------------------------------------
    afr_gt = encontrar_coluna(
        df_original,
        ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"]
    )
    maf_gt = encontrar_coluna(
        df_original,
        ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"]
    )

    co2_ground_truth = (
        compute_co2(afr_gt, maf_gt, fuel)
        if afr_gt is not None and maf_gt is not None
        else None
    )

    # ------------------------------------------------------------------
    # Prediction (Baseline)
    # ------------------------------------------------------------------
    co2_baseline = compute_co2(
        modelo_afr.predict(df_original[features]),
        modelo_maf.predict(df_original[features]),
        fuel
    )

    # ------------------------------------------------------------------
    # Prediction (Metamorphic)
    # ------------------------------------------------------------------
    co2_metamorphic = compute_co2(
        modelo_afr.predict(df_modificado[features]),
        modelo_maf.predict(df_modificado[features]),
        fuel
    )

    # ------------------------------------------------------------------
    # Métricas
    # ------------------------------------------------------------------
    rmse = mean_squared_error(co2_baseline, co2_metamorphic, squared=False)
    pearson_baseline = pearson_safe(co2_baseline, co2_metamorphic)
    pearson_gt = pearson_safe(co2_ground_truth, co2_metamorphic)

    # ======================================================
    # 📋 LOCAL INSPECTION TABLE (LEVEL 1)
    # ======================================================
    tabela = pd.concat([
    df_original[features].iloc[:10].add_suffix("_Baseline"),
    pd.DataFrame({"→": ["→"] * 10}),
    df_modificado[features].iloc[:10].add_suffix("_Metamorphic"),
    pd.DataFrame({
        "CO2_Ground_Truth": co2_ground_truth[:10] if co2_ground_truth is not None else "N/A",
        "CO2_Pred_Baseline": co2_baseline[:10],
        "CO2_Pred_Metamorphic": co2_metamorphic[:10]
    })
    ], axis=1).fillna("N/A")

    from IPython.display import display
    display(
    tabela.style.set_caption(
        f"Local inspection – {nome_ct} ({fuel.upper()})"
    )
)

       
       # ======================================================
    # 📢 BLOCO INFORMATIVO (ANTES DO GRÁFICO)
    # ======================================================
    print("\n" + "=" * 80)
    print(f"🧪 {nome_ct} ({fuel.upper()})")
    print("Metamorphic Test – Level 1 (Sensitivity Analysis)")
    print(f"RMSE Prediction (Baseline) × Prediction (Metamorphic) = {rmse:.4f}")
    print(f"Pearson Prediction (Baseline) × Prediction (Metamorphic) = {pearson_baseline:.3f}")
    print(f"Pearson Ground Truth × Prediction (Metamorphic) = {pearson_gt:.3f}")
    print("=" * 80)

    # ======================================================
    # 📈 GRÁFICO
    # ======================================================
    fig = go.Figure()

    if co2_ground_truth is not None:
        fig.add_trace(go.Scatter(
            y=co2_ground_truth[:200],
            name="Ground Truth",
            line=dict(dash="dot", color="gray")
        ))

    fig.add_trace(go.Scatter(
        y=co2_baseline[:200],
        name="Prediction (Baseline)",
        line=dict(dash="dash", color="blue")
    ))

    fig.add_trace(go.Scatter(
        y=co2_metamorphic[:200],
        name="Prediction (Metamorphic)",
        line=dict(color="red")
    ))

    fig.update_layout(
        title=dict(
    text=(
        f"Local inspection – {nome_ct}<br>"
        f"RMSE={rmse:.3f} | Pearson(Baseline×Metamorphic)={pearson_baseline:.3f}"
    ),
    x=0.5
),
xaxis_title="Samples",
yaxis_title="Output Variable (E)",
template="plotly_white"
)


    fig.show()


# ==============================================================================
# 🧪 METAMORPHIC TEST – NULL ROBUSTNESS (LEVEL 1)
# ==============================================================================

def executar_teste_null(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_ct):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for f in features:
        if f not in df_original.columns:
            df_original[f] = 0
        if f not in df_modificado.columns:
            df_modificado[f] = 0

    # ------------------------------------------------------------------
    # Ground Truth
    # ------------------------------------------------------------------
    afr_gt = encontrar_coluna(
        df_original,
        ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"]
    )
    maf_gt = encontrar_coluna(
        df_original,
        ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"]
    )

    co2_ground_truth = (
        compute_co2(afr_gt, maf_gt, fuel)
        if afr_gt is not None and maf_gt is not None
        else None
    )

    # ------------------------------------------------------------------
    # Prediction (Baseline)
    # ------------------------------------------------------------------
    co2_baseline = compute_co2(
        modelo_afr.predict(df_original[features]),
        modelo_maf.predict(df_original[features]),
        fuel
    )

    # ------------------------------------------------------------------
    # Prediction (Metamorphic – NULL)
    # ------------------------------------------------------------------
    co2_metamorphic = compute_co2(
        modelo_afr.predict(df_modificado[features]),
        modelo_maf.predict(df_modificado[features]),
        fuel
    )

    # ------------------------------------------------------------------
    # Metrics
    # ------------------------------------------------------------------
    rmse = mean_squared_error(co2_baseline, co2_metamorphic, squared=False)
    pearson_baseline = pearson_safe(co2_baseline, co2_metamorphic)
    pearson_gt = pearson_safe(co2_ground_truth, co2_metamorphic)

    # ======================================================
    # 📢 LOG
    # ======================================================
    print("\n" + "=" * 80)
    print(f"🧪 {nome_ct} ({fuel.upper()})")
    print("Metamorphic Test – Level 1 (NULL Robustness)")
    print(f"RMSE Prediction (Baseline) × Prediction (Metamorphic) = {rmse:.4f}")
    print(f"Pearson Prediction (Baseline) × Prediction (Metamorphic) = {pearson_baseline:.3f}")
    print(f"Pearson Ground Truth × Prediction (Metamorphic) = {pearson_gt:.3f}")
    print("=" * 80)


    
    # ======================================================
    # 📋 LOCAL INSPECTION TABLE – NULL ROBUSTNESS (LEVEL 1)
    # ======================================================
    tabela = pd.concat(
        [
            df_original[features].iloc[:10].add_suffix("_Baseline"),
            pd.DataFrame({" ": ["→"] * 10}),
            df_modificado[features].iloc[:10].add_suffix("_NULL"),
            pd.DataFrame({" ": [" "] * 10}),
            pd.DataFrame(
                {
                    "CO2_Ground_Truth": (
                        co2_ground_truth[:10]
                        if co2_ground_truth is not None
                        else ["N/A"] * 10
                    ),
                    "CO2_Pred_Baseline": co2_baseline[:10],
                    "CO2_Pred_NULL": co2_metamorphic[:10],
                }
            ),
        ],
        axis=1,
    ).fillna("NULL")

    from IPython.display import display

    display(
        tabela.style.set_caption(
            f"Local Inspection – NULL Robustness | {nome_ct} ({fuel.upper()})"
        )
    )

    

    # ======================================================
    # 📈 PLOT
    # ======================================================
    fig = go.Figure()

    if co2_ground_truth is not None:
        fig.add_trace(go.Scatter(
            y=co2_ground_truth[:200],
            name="Ground Truth",
            line=dict(dash="dot", color="gray")
        ))

    fig.add_trace(go.Scatter(
        y=co2_baseline[:200],
        name="Prediction (Baseline)",
        line=dict(dash="dash", color="blue")
    ))

    fig.add_trace(go.Scatter(
        y=co2_metamorphic[:200],
        name="Prediction (Metamorphic)",
        line=dict(color="red")
    ))

    fig.update_layout(
        title=dict(
    text=(
        f"Local inspection – {nome_ct}<br>"
        f"RMSE={rmse:.3f} | Pearson(Baseline×Metamorphic)={pearson_baseline:.3f}"
    ),
    x=0.5
 ) 
,
        xaxis_title="Samples",
        yaxis_title="Output Variable (E)",
        template="plotly_white"
    )

    fig.show()


# ======================================================================
# 🔧 INTERNAL VARIABLE MAPPING (LEVEL 1 – BLACK-BOX)
# A, B, C, D are abstract input variables (implementation-only mapping)
# ======================================================================

variaveis_n1 = {
    "A": "Speed(OBD)(km/h)",
    "B": "Acceleration",
    "C": "Latitude",
    "D": "Longitude"
}


# ==============================================================================
# 🔁 TEST CASES – LEVEL 1
# ==============================================================================


testes_exclusivos = {
    "CT_1A_001 – A = 0": lambda df: df.assign(**{"Speed(OBD)(km/h)": 0.0}),
    "CT_1A_002 – B = 0": lambda df: df.assign(Acceleration=0.0),
    "CT_1A_003 – C = 0": lambda df: df.assign(Latitude=0.0),
    "CT_1A_004 – D = 0": lambda df: df.assign(Longitude=0.0),
}

testes_null = {
    "CT_1B_001 – A = NULL": lambda df: df.assign(**{"Speed(OBD)(km/h)": np.nan}),
    "CT_1B_002 – B = NULL": lambda df: df.assign(Acceleration=np.nan),
    "CT_1B_003 – C = NULL": lambda df: df.assign(Latitude=np.nan),
    "CT_1B_004 – D = NULL": lambda df: df.assign(Longitude=np.nan),
}

lags = [10, 50, 100]


testes_temporal_lag = {
    f"CT_1C_{idx:03d} – [Temporal] {var} Lag (lag = {lag})": (
        lambda v=col, l=lag: lambda df: df.assign(**{v: df[v].shift(l)})
    )()
    for idx, (var, col, lag) in enumerate(
        [(var, col, lag) for var, col in variaveis_n1.items() for lag in lags], start=1
    )
}


testes_multiplicativos = {
    f"CT_1D_{idx:03d} – [Multiplicative] {var} Scaling (×20)":
        (lambda v=col:
            lambda df: df.assign(**{v: df[v] * 20})
        )()
    for idx, (var, col) in enumerate(
        variaveis_n1.items(),
        start=1
    )
}    

np.random.seed(42)

sigma_ruido = 0.2  # acima de sigma pequeno (ruído moderado)

testes_ruido_gaussiano = {
    f"CT_1E_{idx:03d} – [Noise] {var} Gaussian Noise (σ={sigma_ruido})":
        (lambda v=col:
            lambda df: df.assign(
                **{
                    v: df[v] + np.random.normal(
                        loc=0.0,
                        scale=sigma_ruido,
                        size=len(df)
                    )
                }
            )
        )()
    for idx, (var, col) in enumerate(
        variaveis_n1.items(),
        start=1
    )
}

Arquivos de dados carregados com sucesso.
Modelos carregados com sucesso.


# ▶️ EXECUTION

##  Zero Value Tests

In [8]:
print("\n=== LEVEL 1 – Variables equal to Zero - METAMORPHIC TESTS ===")
for nome, t in testes_exclusivos.items():
    for fuel, module_name, df in [
        ("ethanol", "Module 1", df_ethanol),
        ("gasoline", "Module 2", df_gasoline)
    ]:
        executar_teste_plotly(
            df_original=df,
            df_modificado=t(df.copy()),
            modelo_afr=modelos[fuel]["afr"],
            modelo_maf=modelos[fuel]["maf"],
            fuel=fuel,                 
            nome_ct=f"{nome} – {module_name}"
        )



=== LEVEL 1 – Variables equal to Zero - METAMORPHIC TESTS ===


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,0.000000,0.000000,0.642839,0.742631,0.727468
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,0.000000,0.555556,0.653249,0.771505,0.727468
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,0.000000,0.000000,0.653249,0.771505,0.727468
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,0.000000,-0.555556,0.679275,0.659013,0.721582
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0.000000,-0.833333,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.000000,0.000000,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.000000,0.000000,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,0.000000,0.833333,1.090483,1.137803,0.706858
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,0.000000,0.000000,1.090483,1.137803,0.706858
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,0.000000,0.000000,0.921315,1.137803,0.706858



🧪 CT_1A_001 – A = 0 – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.4900
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.395
Pearson Ground Truth × Prediction (Metamorphic) = 0.406


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,0.000000,0.000000,1.002671,1.456781,0.965052
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,0.000000,-0.277778,1.013360,1.016708,0.884055
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,0.000000,0.277778,2.088720,1.456781,0.965052
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,0.000000,0.555556,2.261889,1.996380,0.965052
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,0.000000,0.555556,2.499195,2.400486,0.940914
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,0.000000,0.833333,2.606089,2.400486,0.940914
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,0.000000,0.833333,3.070012,2.148721,0.880813
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,0.000000,0.833333,2.445747,1.470940,0.773157
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,0.000000,0.000000,1.073221,1.588029,0.783680
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,0.000000,-0.555556,0.562265,1.085921,0.980167



🧪 CT_1A_001 – A = 0 – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.8179
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.639
Pearson Ground Truth × Prediction (Metamorphic) = 0.537


...


🧪 CT_1A_003 – C = 0 – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 1.0747
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.030
Pearson Ground Truth × Prediction (Metamorphic) = 0.020


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,0.000000,3,0.000000,0.642839,0.742631,0.665039
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,0.000000,5,0.555556,0.653249,0.771505,0.581781
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,0.000000,5,0.000000,0.653249,0.771505,0.581781
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,0.000000,3,-0.555556,0.679275,0.659013,0.617370
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,0.000000,0,-0.833333,0.610306,0.667689,0.466419
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,0.000000,0,0.000000,0.610306,0.706858,0.474772
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,0.000000,0,0.000000,0.490587,0.706858,0.474772
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,0.000000,3,0.833333,1.090483,1.137803,0.665039
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,0.000000,3,0.000000,1.090483,1.137803,0.665039
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,0.000000,3,0.000000,0.921315,1.137803,0.665039



🧪 CT_1A_004 – D = 0 – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.4241
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.385
Pearson Ground Truth × Prediction (Metamorphic) = 0.351


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,0.000000,5,0.000000,1.002671,1.456781,0.354376
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,0.000000,4,-0.277778,1.013360,1.016708,0.631768
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,0.000000,5,0.277778,2.088720,1.456781,0.354376
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,0.000000,7,0.555556,2.261889,1.996380,0.574013
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,0.000000,9,0.555556,2.499195,2.400486,0.818026
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,0.000000,12,0.833333,2.606089,2.400486,0.653651
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,0.000000,15,0.833333,3.070012,2.148721,0.505126
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,0.000000,18,0.833333,2.445747,1.470940,0.484962
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,0.000000,18,0.000000,1.073221,1.588029,0.481107
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,0.000000,16,-0.555556,0.562265,1.085921,0.750878



🧪 CT_1A_004 – D = 0 – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.7443
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.456
Pearson Ground Truth × Prediction (Metamorphic) = 0.398


##  Null Value Tests

In [9]:
print("\n=== LEVEL 1 – Variable equal to NULL Value - METAMORPHIC TESTS ===")
for nome, t in testes_null.items():
    for fuel, module_name, df in [
        ("ethanol", "Module 1", df_ethanol),
        ("gasoline", "Module 2", df_gasoline)
    ]:
        executar_teste_null(
            df_original=df,
            df_modificado=t(df.copy()),
            modelo_afr=modelos[fuel]["afr"],
            modelo_maf=modelos[fuel]["maf"],
            fuel=fuel,
            nome_ct=f"{nome} – {module_name}"
        )


=== LEVEL 1 – Variable equal to NULL Value - METAMORPHIC TESTS ===

🧪 CT_1B_001 – A = NULL – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (NULL Robustness)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.9639
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.391
Pearson Ground Truth × Prediction (Metamorphic) = 0.347


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_NULL,Longitude_NULL,Speed(OBD)(km/h)_NULL,Acceleration_NULL,,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_NULL
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,NULL,0.000000,,0.642839,0.742631,1.573511
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,NULL,0.555556,,0.653249,0.771505,1.573511
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,NULL,0.000000,,0.653249,0.771505,1.573511
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,NULL,-0.555556,,0.679275,0.659013,1.428636
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,NULL,-0.833333,,0.610306,0.667689,1.366803
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,NULL,0.000000,,0.610306,0.706858,1.514831
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,NULL,0.000000,,0.490587,0.706858,1.514831
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,NULL,0.833333,,1.090483,1.137803,1.514831
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,NULL,0.000000,,1.090483,1.137803,1.514831
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,NULL,0.000000,,0.921315,1.137803,1.514831



🧪 CT_1B_001 – A = NULL – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (NULL Robustness)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 1.2611
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.493
Pearson Ground Truth × Prediction (Metamorphic) = 0.391


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_NULL,Longitude_NULL,Speed(OBD)(km/h)_NULL,Acceleration_NULL,,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_NULL
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,NULL,0.000000,,1.002671,1.456781,3.653032
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,NULL,-0.277778,,1.013360,1.016708,3.031917
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,NULL,0.277778,,2.088720,1.456781,3.653032
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,NULL,0.555556,,2.261889,1.996380,3.653032
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,NULL,0.555556,,2.499195,2.400486,3.134111
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,NULL,0.833333,,2.606089,2.400486,3.134111
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,NULL,0.833333,,3.070012,2.148721,3.033430
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,NULL,0.833333,,2.445747,1.470940,2.350986
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,NULL,0.000000,,1.073221,1.588029,2.350986
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,NULL,-0.555556,,0.562265,1.085921,2.351204



🧪 CT_1B_002 – B = NULL – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (NULL Robustness)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.1949
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.900
Pearson Ground Truth × Prediction (Metamorphic) = 0.752


...

,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_NULL,Longitude_NULL,Speed(OBD)(km/h)_NULL,Acceleration_NULL,,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_NULL
0,-5.820010,-35.220825,5,0.000000,→,NULL,-35.220825,5,0.000000,,1.002671,1.456781,2.210472
1,-5.820013,-35.220822,4,-0.277778,→,NULL,-35.220822,4,-0.277778,,1.013360,1.016708,2.381479
2,-5.820016,-35.220819,5,0.277778,→,NULL,-35.220819,5,0.277778,,2.088720,1.456781,2.210472
3,-5.820014,-35.220825,7,0.555556,→,NULL,-35.220825,7,0.555556,,2.261889,1.996380,2.764091
4,-5.819992,-35.220829,9,0.555556,→,NULL,-35.220829,9,0.555556,,2.499195,2.400486,2.733665
5,-5.819974,-35.220824,12,0.833333,→,NULL,-35.220824,12,0.833333,,2.606089,2.400486,2.900924
6,-5.819944,-35.220823,15,0.833333,→,NULL,-35.220823,15,0.833333,,3.070012,2.148721,2.524258
7,-5.819908,-35.220810,18,0.833333,→,NULL,-35.220810,18,0.833333,,2.445747,1.470940,2.125705
8,-5.819866,-35.220794,18,0.000000,→,NULL,-35.220794,18,0.000000,,1.073221,1.588029,2.241735
9,-5.819821,-35.220775,16,-0.555556,→,NULL,-35.220775,16,-0.555556,,0.562265,1.085921,1.960367



🧪 CT_1B_004 – D = NULL – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (NULL Robustness)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.4241
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.385
Pearson Ground Truth × Prediction (Metamorphic) = 0.351


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_NULL,Longitude_NULL,Speed(OBD)(km/h)_NULL,Acceleration_NULL,,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_NULL
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,NULL,3,0.000000,,0.642839,0.742631,0.665039
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,NULL,5,0.555556,,0.653249,0.771505,0.581781
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,NULL,5,0.000000,,0.653249,0.771505,0.581781
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,NULL,3,-0.555556,,0.679275,0.659013,0.617370
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,NULL,0,-0.833333,,0.610306,0.667689,0.466419
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,NULL,0,0.000000,,0.610306,0.706858,0.474772
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,NULL,0,0.000000,,0.490587,0.706858,0.474772
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,NULL,3,0.833333,,1.090483,1.137803,0.665039
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,NULL,3,0.000000,,1.090483,1.137803,0.665039
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,NULL,3,0.000000,,0.921315,1.137803,0.665039



🧪 CT_1B_004 – D = NULL – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (NULL Robustness)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.7443
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.456
Pearson Ground Truth × Prediction (Metamorphic) = 0.398


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_NULL,Longitude_NULL,Speed(OBD)(km/h)_NULL,Acceleration_NULL,,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_NULL
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,NULL,5,0.000000,,1.002671,1.456781,0.354376
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,NULL,4,-0.277778,,1.013360,1.016708,0.631768
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,NULL,5,0.277778,,2.088720,1.456781,0.354376
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,NULL,7,0.555556,,2.261889,1.996380,0.574013
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,NULL,9,0.555556,,2.499195,2.400486,0.818026
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,NULL,12,0.833333,,2.606089,2.400486,0.653651
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,NULL,15,0.833333,,3.070012,2.148721,0.505126
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,NULL,18,0.833333,,2.445747,1.470940,0.484962
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,NULL,18,0.000000,,1.073221,1.588029,0.481107
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,NULL,16,-0.555556,,0.562265,1.085921,0.750878


##  Temporal Lag Tests

In [10]:
print("\n=== LEVEL 1 – Temporal Lag  METAMORPHIC TESTS ===")
for nome, t in testes_temporal_lag.items():
    for fuel, module_name, df in [
        ("ethanol", "Module 1", df_ethanol),
        ("gasoline", "Module 2", df_gasoline)
    ]:
        executar_teste_plotly(
            df_original=df,
            df_modificado=t(df.copy()),
            modelo_afr=modelos[fuel]["afr"],
            modelo_maf=modelos[fuel]["maf"],
            fuel=fuel,
            nome_ct=f"{nome} – {module_name}"
        )


=== LEVEL 1 – Temporal Lag  METAMORPHIC TESTS ===


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,N/A,0.000000,0.642839,0.742631,1.573511
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,N/A,0.555556,0.653249,0.771505,1.573511
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,N/A,0.000000,0.653249,0.771505,1.573511
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,N/A,-0.555556,0.679275,0.659013,1.428636
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,N/A,-0.833333,0.610306,0.667689,1.366803
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,N/A,0.000000,0.610306,0.706858,1.514831
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,N/A,0.000000,0.490587,0.706858,1.514831
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,N/A,0.833333,1.090483,1.137803,1.514831
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,N/A,0.000000,1.090483,1.137803,1.514831
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,N/A,0.000000,0.921315,1.137803,1.514831



🧪 CT_1C_001 – [Temporal] A Lag (lag = 10) – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.2870
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.777
Pearson Ground Truth × Prediction (Metamorphic) = 0.625


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,N/A,0.000000,1.002671,1.456781,3.653032
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,N/A,-0.277778,1.013360,1.016708,3.031917
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,N/A,0.277778,2.088720,1.456781,3.653032
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,N/A,0.555556,2.261889,1.996380,3.653032
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,N/A,0.555556,2.499195,2.400486,3.134111
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,N/A,0.833333,2.606089,2.400486,3.134111
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,N/A,0.833333,3.070012,2.148721,3.033430
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,N/A,0.833333,2.445747,1.470940,2.350986
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,N/A,0.000000,1.073221,1.588029,2.350986
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,N/A,-0.555556,0.562265,1.085921,2.351204



🧪 CT_1C_001 – [Temporal] A Lag (lag = 10) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.4046
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.853
Pearson Ground Truth × Prediction (Metamorphic) = 0.676


...


🧪 CT_1C_011 – [Temporal] D Lag (lag = 50) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.7595
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.541
Pearson Ground Truth × Prediction (Metamorphic) = 0.434


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,N/A,3,0.000000,0.642839,0.742631,0.665039
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,N/A,5,0.555556,0.653249,0.771505,0.581781
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,N/A,5,0.000000,0.653249,0.771505,0.581781
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,N/A,3,-0.555556,0.679275,0.659013,0.617370
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,N/A,0,-0.833333,0.610306,0.667689,0.466419
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,N/A,0,0.000000,0.610306,0.706858,0.474772
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,N/A,0,0.000000,0.490587,0.706858,0.474772
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,N/A,3,0.833333,1.090483,1.137803,0.665039
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,N/A,3,0.000000,1.090483,1.137803,0.665039
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,N/A,3,0.000000,0.921315,1.137803,0.665039



🧪 CT_1C_012 – [Temporal] D Lag (lag = 100) – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.3921
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.578
Pearson Ground Truth × Prediction (Metamorphic) = 0.499


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,N/A,5,0.000000,1.002671,1.456781,0.354376
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,N/A,4,-0.277778,1.013360,1.016708,0.631768
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,N/A,5,0.277778,2.088720,1.456781,0.354376
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,N/A,7,0.555556,2.261889,1.996380,0.574013
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,N/A,9,0.555556,2.499195,2.400486,0.818026
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,N/A,12,0.833333,2.606089,2.400486,0.653651
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,N/A,15,0.833333,3.070012,2.148721,0.505126
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,N/A,18,0.833333,2.445747,1.470940,0.484962
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,N/A,18,0.000000,1.073221,1.588029,0.481107
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,N/A,16,-0.555556,0.562265,1.085921,0.750878



🧪 CT_1C_012 – [Temporal] D Lag (lag = 100) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.7636
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.546
Pearson Ground Truth × Prediction (Metamorphic) = 0.449


##  Multiplicative Tests

In [11]:
print("\n=== LEVEL 1 – MULTIPLICATIVE SCALING (×20) – METAMORPHIC TESTS ===")

for nome, t in testes_multiplicativos.items():
    for fuel, module_name, df in [
        ("ethanol", "Module 1", df_ethanol),
        ("gasoline", "Module 2", df_gasoline)
    ]:
        executar_teste_plotly(
            df_original=df,
            df_modificado=t(df.copy()),
            modelo_afr=modelos[fuel]["afr"],
            modelo_maf=modelos[fuel]["maf"],
            fuel=fuel,
            nome_ct=f"{nome} – {module_name}"
        )


=== LEVEL 1 – MULTIPLICATIVE SCALING (×20) – METAMORPHIC TESTS ===


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,60,0.000000,0.642839,0.742631,1.006023
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,100,0.555556,0.653249,0.771505,1.623527
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,100,0.000000,0.653249,0.771505,1.579490
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,60,-0.555556,0.679275,0.659013,0.914030
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0,-0.833333,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,0.000000,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,0.000000,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,60,0.833333,1.090483,1.137803,1.000498
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,60,0.000000,1.090483,1.137803,0.924211
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,60,0.000000,0.921315,1.137803,0.924211



🧪 CT_1D_001 – [Multiplicative] A Scaling (×20) – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.6804
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.673
Pearson Ground Truth × Prediction (Metamorphic) = 0.575


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,100,0.000000,1.002671,1.456781,3.189422
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,80,-0.277778,1.013360,1.016708,2.051800
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,100,0.277778,2.088720,1.456781,3.550872
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,140,0.555556,2.261889,1.996380,3.550872
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,180,0.555556,2.499195,2.400486,3.046464
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,240,0.833333,2.606089,2.400486,3.046464
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,300,0.833333,3.070012,2.148721,2.948598
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,360,0.833333,2.445747,1.470940,2.285239
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,360,0.000000,1.073221,1.588029,2.052619
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,320,-0.555556,0.562265,1.085921,1.591138



🧪 CT_1D_001 – [Multiplicative] A Scaling (×20) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.6509
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.827
Pearson Ground Truth × Prediction (Metamorphic) = 0.680


...


🧪 CT_1D_003 – [Multiplicative] C Scaling (×20) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.7673
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.486
Pearson Ground Truth × Prediction (Metamorphic) = 0.396


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-704.416356,3,0.000000,0.642839,0.742631,0.742631
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-704.416556,5,0.555556,0.653249,0.771505,0.771505
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-704.416752,5,0.000000,0.653249,0.771505,0.771505
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-704.416859,3,-0.555556,0.679275,0.659013,0.659013
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-704.417003,0,-0.833333,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-704.417003,0,0.000000,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-704.417003,0,0.000000,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-704.417113,3,0.833333,1.090483,1.137803,1.137803
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-704.417188,3,0.000000,1.090483,1.137803,1.137803
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-704.417156,3,0.000000,0.921315,1.137803,1.137803



🧪 CT_1D_004 – [Multiplicative] D Scaling (×20) – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.6111
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.375
Pearson Ground Truth × Prediction (Metamorphic) = 0.334


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-704.416499,5,0.000000,1.002671,1.456781,1.456781
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-704.416450,4,-0.277778,1.013360,1.016708,1.016708
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-704.416388,5,0.277778,2.088720,1.456781,1.456781
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-704.416492,7,0.555556,2.261889,1.996380,1.996380
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-704.416575,9,0.555556,2.499195,2.400486,2.400486
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-704.416484,12,0.833333,2.606089,2.400486,2.400486
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-704.416462,15,0.833333,3.070012,2.148721,2.148721
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-704.416208,18,0.833333,2.445747,1.470940,1.470940
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-704.415882,18,0.000000,1.073221,1.588029,1.470562
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-704.415490,16,-0.555556,0.562265,1.085921,1.085921



🧪 CT_1D_004 – [Multiplicative] D Scaling (×20) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.9966
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.478
Pearson Ground Truth × Prediction (Metamorphic) = 0.390


##  Gaussian Noise Injection Tests

In [12]:
print("\n=== LEVEL 1 – Gaussian Noise Injection (σ = 0.2) ===")
for nome, t in testes_ruido_gaussiano.items():
    for fuel, module_name, df in [
        ("ethanol", "Module 1", df_ethanol),
        ("gasoline", "Module 2", df_gasoline)
    ]:
        executar_teste_plotly(
            df_original=df,
            df_modificado=t(df.copy()),
            modelo_afr=modelos[fuel]["afr"],
            modelo_maf=modelos[fuel]["maf"],
            fuel=fuel,
            nome_ct=f"{nome} – {module_name}"
        )


=== LEVEL 1 – Gaussian Noise Injection (σ = 0.2) ===


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,3.099343,0.000000,0.642839,0.742631,0.742631
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,4.972347,0.555556,0.653249,0.771505,0.764202
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,5.129538,0.000000,0.653249,0.771505,0.771505
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,3.304606,-0.555556,0.679275,0.659013,0.659013
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,-0.046831,-0.833333,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,-0.046827,0.000000,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.315843,0.000000,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,3.153487,0.833333,1.090483,1.137803,1.137803
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,2.906105,0.000000,1.090483,1.137803,0.849757
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,3.108512,0.000000,0.921315,1.137803,1.137803



🧪 CT_1E_001 – [Noise] A Gaussian Noise (σ=0.2) – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.0565
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.992
Pearson Ground Truth × Prediction (Metamorphic) = 0.815


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,4.809212,0.000000,1.002671,1.456781,1.456781
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,3.918593,-0.277778,1.013360,1.016708,0.886840
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,5.137264,0.277778,2.088720,1.456781,1.456781
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,7.021179,0.555556,2.261889,1.996380,1.996380
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,9.116883,0.555556,2.499195,2.400486,2.400486
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,12.395288,0.833333,2.606089,2.400486,2.400486
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,14.687152,0.833333,3.070012,2.148721,2.390285
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,18.323443,0.833333,2.445747,1.470940,1.470940
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,18.020871,0.000000,1.073221,1.588029,1.588029
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,15.820243,-0.555556,0.562265,1.085921,1.140141



🧪 CT_1E_001 – [Noise] A Gaussian Noise (σ=0.2) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.0670
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.996
Pearson Ground Truth × Prediction (Metamorphic) = 0.805


...


🧪 CT_1E_003 – [Noise] C Gaussian Noise (σ=0.2) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.9360
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.253
Pearson Ground Truth × Prediction (Metamorphic) = 0.194


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.375894,3,0.000000,0.642839,0.742631,0.742631
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.359990,5,0.555556,0.653249,0.771505,0.771505
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.252703,5,0.000000,0.653249,0.771505,0.771505
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-34.863931,3,-0.555556,0.679275,0.659013,0.617370
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.253935,0,-0.833333,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.486352,0,0.000000,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.321086,0,0.000000,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.069444,3,0.833333,1.090483,1.137803,0.665039
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.082871,3,0.000000,1.090483,1.137803,0.665039
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.260901,3,0.000000,0.921315,1.137803,1.137803



🧪 CT_1E_004 – [Noise] D Gaussian Noise (σ=0.2) – Module 1 (ETHANOL)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.5078
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.340
Pearson Ground Truth × Prediction (Metamorphic) = 0.302


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_Ground_Truth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.360848,5,0.000000,1.002671,1.456781,1.456781
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.459166,4,-0.277778,1.013360,1.016708,1.016708
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.262521,5,0.277778,2.088720,1.456781,1.456781
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.327180,7,0.555556,2.261889,1.996380,1.996380
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.101569,9,0.555556,2.499195,2.400486,0.818026
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.231103,12,0.833333,2.606089,2.400486,2.400486
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.095283,15,0.833333,3.070012,2.148721,0.505126
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.279015,18,0.833333,2.445747,1.470940,1.470940
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.353816,18,0.000000,1.073221,1.588029,1.470562
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.337487,16,-0.555556,0.562265,1.085921,1.085921



🧪 CT_1E_004 – [Noise] D Gaussian Noise (σ=0.2) – Module 2 (GASOLINE)
Metamorphic Test – Level 1 (Sensitivity Analysis)
RMSE Prediction (Baseline) × Prediction (Metamorphic) = 0.8754
Pearson Prediction (Baseline) × Prediction (Metamorphic) = 0.432
Pearson Ground Truth × Prediction (Metamorphic) = 0.341
